# Topik: Analisis Sentimen Opini Publik terhadap Kebijakan Pemindahan Dana Rp 200 Triliun ke Bank Himbara oleh Menteri Keuangan Purbaya melalui Komentar YouTube

## Notebook: Preprocessing Data dari data hasil filtering.

**Anggota Kelompok:**  
- 140810230011 - Lukas Austin  
- 140810230045 - Devin Suryadi  
- 140810230057 - Orlando Bloem Sutono  

Mata Kuliah: Data Mining  
Kelas: A  
Dosen: Bu Helen

---

## 1. Import Library
Library yang dibutuhkan untuk preprocessing teks.

In [17]:
import os
import re
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import emoji
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## 2. Load Dataset
Memuat data hasil filtering komentar dari YouTube.

In [18]:
df = pd.read_csv("dataset_filtered.csv")
print(f"Dataset dimuat: {len(df)} baris, kolom: {list(df.columns)}")

df = df[['comment']].copy()
df.head()

Dataset dimuat: 924 baris, kolom: ['id_comment', 'comment', 'author', 'like_count', 'time', 'video_id']


,comment
0,"Seperti langit dan bumi, kerja pak Purbaya dan..."
1,"Semoga Purbaya selamat, ngak dikerjain sama ol..."
2,nyatanya bank2 himbara masih selektif mengucur...
3,Ayooo pak pur ttp semangat bangun indonesia 👍
4,Ayo rakyat cerdas harus mengonyrol pemaik peng...


## 3. Text Cleaning
Melakukan pembersihan HTML, URL, emoji, angka, tanda baca, dan karakter berulang.

In [19]:
# Hapus HTML dan tag tidak relevan
df['clean_text'] = df['comment'].apply(lambda x: BeautifulSoup(str(x), "html.parser").get_text())

# Normalisasi unicode
df['clean_text'] = df['clean_text'].apply(lambda x: unicodedata.normalize('NFKC', str(x)))

# Hapus URL, mention, hashtag
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r"http\S+|www\S+|@\w+|#\w+", " ", x))

# Hapus emoji
df['clean_text'] = df['clean_text'].apply(lambda x: emoji.replace_emoji(x, replace=' '))

# Lowercase dan hapus angka/tanda baca
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', ' ', x.lower()))

# Normalisasi spasi dan huruf berulang
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'(.)\1{2,}', r'\1\1', x))
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

df[['comment', 'clean_text']].head()


,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan...",seperti langit dan bumi kerja pak purbaya dan ...
1,"Semoga Purbaya selamat, ngak dikerjain sama ol...",semoga purbaya selamat ngak dikerjain sama oli...
2,nyatanya bank2 himbara masih selektif mengucur...,nyatanya bank himbara masih selektif mengucurk...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayoo pak pur ttp semangat bangun indonesia
4,Ayo rakyat cerdas harus mengonyrol pemaik peng...,ayo rakyat cerdas harus mengonyrol pemaik peng...


## 4. Slang Normalization
Mengganti kata tidak baku (gaul) dengan kata yang benar.

In [20]:
# Dictionary untuk kata slang
slang_dict = {
    # Bahasa tidak baku umum
    "gk": "tidak", "ga": "tidak", "gak": "tidak", "nggak": "tidak", "ngga": "tidak",
    "bgt": "banget", "yg": "yang", "klo": "kalau", "kalo": "kalau", "tp": "tetapi",
    "trs": "terus", "br": "baru", "kpn": "kapan", "pls": "tolong", "makasih": "terima kasih",
    "dl": "dulu", "aja": "saja", "sm": "sama", "jd": "jadi", "dr": "dari", "nih": "ini",
    "gw": "saya", "gue": "saya", "gua": "saya", "lu": "kamu", "loe": "kamu", "elo": "kamu",
    "utk": "untuk", "dgn": "dengan", "dg": "dengan", "tdk": "tidak", "jgn": "jangan",
    "sdh": "sudah", "udh": "sudah", "krn": "karena", "karna": "karena", "bpk": "bapak",
    "tau": "tahu", "sy": "saya", "bikin": "buat", "kasih": "beri", "blm": "belum",
    "skrg": "sekarang", "knp": "kenapa", "byk": "banyak", "msh": "masih", "dpt": "dapat",
    "tpn": "tetapi", "bgtu": "begitu", "ngapain": "untuk apa", "jdgn": "jangan", "kl": "kalau",
    "td": "tadi", "skrng": "sekarang", "brp": "berapa", "bbrp": "beberapa",
    "gmn": "bagaimana", "gimana": "bagaimana", "utkmu": "untuk kamu", "smua": "semua", "nnti": "nanti",
    "ttp": "tetap", "sblm": "sebelum", "smg": "semoga",

    # Istilah ekonomi dan pemerintahan
    "menkeu": "menteri keuangan", "mentri": "menteri", "resiko": "risiko",
    "duit": "uang", "bi": "bank indonesia", "kur": "kredit usaha rakyat",
    "pinjol": "pinjaman online", "bumn": "badan usaha milik negara",
    "umkm": "usaha mikro kecil menengah", "dpr": "dewan perwakilan rakyat",
    "pmerintah": "pemerintah", "pemerintahn": "pemerintahan", "krdit": "kredit",
    "bri": "bank rakyat indonesia", "himbara": "himpunan bank milik negara",
    "menku": "menteri keuangan", "pur" : "purbaya",

    # Ekspresi emosi umum
    "mantappp": "mantap", "mantapp": "mantap", "mantab": "mantap", "bangett": "banget",
    "semangattt": "semangat", "parahh": "parah", "baguss": "bagus", "jelekk": "jelek",
    "luarbiasa": "luar biasa", "mantappu": "mantap", "kereenn": "keren", "oke": "baik",
    "ok": "baik", "sip": "baik", "wow": "luar biasa", "anjir": "", "anjay": "", "astaga": "",
    "waduh": "", "yaampun": "", "yah": "", "uhh": "", "hehe": "", "hahaha": "",
    "lol": "", "rip": "", "btw": "ngomong ngomong", "thx": "terima kasih", "tq": "terima kasih",

    # Kata penegas khas komentar
    "pak": "bapak", "bpk.": "bapak", "bu": "ibu", "bunda": "ibu", "bpk": "bapak",
    "bapake": "bapak", "ibunya": "ibu", "bapaknya": "bapak", "mas": "saudara",
    "mba": "saudari", "mbak": "saudari", "om": "saudara", "bang": "saudara",

    # Penulisan tidak baku angka atau singkatan umum
    "g": "tidak", "gaes": "teman teman", "smoga": "semoga", "selamet": "selamat",
    "sehattt": "sehat", "plis": "tolong", "mksd": "maksud", "trims": "terima kasih",
    "terimakasih": "terima kasih", "terimakasi": "terima kasih", "tqsm": "terima kasih banyak",

    # Bahasa Inggris umum
    "bro": "saudara", "sis": "saudari", "idk": "tidak tahu",
    "omg": "astaga", "thanks": "terima kasih", "yup": "ya",
    "yups": "ya", "nope": "tidak", "yess": "ya", "yeah": "ya", "woww": "luar biasa",
    "good": "bagus", "nice": "bagus", "great": "bagus", "amazing": "luar biasa",
    "awesome": "luar biasa", "love": "suka", "hate": "benci", "support": "dukung",

    # Tambahan misspelling
    "nyatanya": "kenyataannya", "ngak": "tidak", "padahal": "walaupun begitu",
    "selektif": "pilih pilih", "usah": "usaha", "percetaka": "percetakan",
    "bunganya": "bunga", "berbahaya": "berisiko", "mengonyrol" : "mengontrol",
    "pemaik" : "pemain"
}

In [21]:
def normalize_slang(text):
    words = text.split()
    return " ".join([slang_dict.get(w, w) for w in words])

df['clean_text'] = df['clean_text'].apply(normalize_slang)
df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan...",seperti langit dan bumi kerja bapak purbaya da...
1,"Semoga Purbaya selamat, ngak dikerjain sama ol...",semoga purbaya selamat tidak dikerjain sama ol...
2,nyatanya bank2 himbara masih selektif mengucur...,kenyataannya bank himpunan bank milik negara m...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayoo bapak purbaya tetap semangat bangun indon...
4,Ayo rakyat cerdas harus mengonyrol pemaik peng...,ayo rakyat cerdas harus mengontrol pemain peng...


## 5. Stopword Removal dan Stemming
Menghapus kata umum yang tidak informatif dan mengembalikan kata ke bentuk dasarnya.

In [25]:
# Inisialisasi Stopword
stop_factory = StopWordRemoverFactory()
stop_remover = stop_factory.create_stop_word_remover()

# Inisialisasi Stemmer
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [23]:
# Fungsi proteksi negasi sebelum stemming karena biasanya library ini menghilangkan kata negasi
def protect_negations(text):
    text = re.sub(r"\btidak\b", "NEG-TIDAK", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbukan\b", "NEG-BUKAN", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbelum\b", "NEG-BELUM", text, flags=re.IGNORECASE)
    text = re.sub(r"\bkurang\b", "NEG-KURANG", text, flags=re.IGNORECASE)
    text = re.sub(r"\btanpa\b", "NEG-TANPA", text, flags=re.IGNORECASE)
    return text

# Fungsi pemulihan negasi setelah stemming
def restore_negations(text):
    text = re.sub(r"\bneg-tidak\b", "tidak", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-bukan\b", "bukan", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-belum\b", "belum", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-kurang\b", "kurang", text, flags=re.IGNORECASE)
    text = re.sub(r"\bneg-tanpa\b", "tanpa", text, flags=re.IGNORECASE)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [24]:
# Proteksi kata negasi
df['clean_text'] = df['clean_text'].apply(protect_negations)

# Stopword removal
df['clean_text'] = df['clean_text'].apply(lambda x: stop_remover.remove(x))

# Stemming
df['clean_text'] = df['clean_text'].apply(lambda x: stemmer.stem(x))

# Pulihkan kata negasi
df['clean_text'] = df['clean_text'].apply(restore_negations)

df[['comment', 'clean_text']].head()

,comment,clean_text
0,"Seperti langit dan bumi, kerja pak Purbaya dan...",langit bumi kerja bapak purbaya ibu sri mulyan...
1,"Semoga Purbaya selamat, ngak dikerjain sama ol...",moga purbaya selamat tidak dikerjain sama olig...
2,nyatanya bank2 himbara masih selektif mengucur...,nyata bank himpun bank milik negara pilih pili...
3,Ayooo pak pur ttp semangat bangun indonesia 👍,ayoo bapak purbaya tetap semangat bangun indon...
4,Ayo rakyat cerdas harus mengonyrol pemaik peng...,ayo rakyat cerdas kontrol main guna uang negar...
